<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/dev/model/BLEURT-CSS-GENERATOR-MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.2.0]**

---



# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [18]:
pip install groq

In [1]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 1173, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1173 (delta 26), reused 40 (delta 22), pack-reused 1102
Receiving objects: 100% (1173/1173), 108.86 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (614/614), done.
Updating files: 100% (122/122), done.


### en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset que esta en formato json

### Accedemos a los datos del dataset TEST.json


In [3]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles)


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)


[{'style': 'body { overflow:hidden ; background:rgb(25,35,125) } ', 'selector': 'body'}, {'style': 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'selector': 'div.drop-container'}, {'style': 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'selector': 'div.drop-container'}, {'style': 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'selector': 'div.drop-container'}, {'style': 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'selector': 'div.drop-container'}, {'style': 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'selector': 'div.drop-container'}, {'style': 'div.drop { position:absolute ; top:-25% ; wid

#Creando las referencias para evaluar mediante BLEURT nuestro css generado

In [13]:
allReferences = [styles["style"] for styles in data_styles]
print(allReferences)

['body { overflow:hidden ; background:rgb(25,35,125) } ', 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ', 'div.drop { position:absolute ; top:-25% ; width:100% ; height:100% ; border-radius:100% 5% 100% 100% ; -webkit-transform:rotate(-45deg) ; transform:rotate(-45deg) ; margin:0 ; background:deepskyblue ; -webkit-animation:drip 4s forwards ; animation:drip 4s forwards } ', 'h1 { color:#fff ; position:absolute ; fo

#Creando los candidatos para evaluar mediante BLEURT nuestro css generado

In [88]:
import random
import re

from groq import Groq

client = Groq(
    api_key="AGREGA LA CLABE DE GROQ ALEX",
)

palettes = ["Análoga","Monocromática","Complementaria", "Triádica", "Tetrádica"];
def randomPalette():
  return random.choice(palettes)

def color():
    """
    Genera un color aleatorio en formato CSS hexadecimal (#RRGGBB).
    """
    r = random.randint(0, 255)  # Genera un valor aleatorio para el componente rojo
    g = random.randint(0, 255)  # Genera un valor aleatorio para el componente verde
    b = random.randint(0, 255)  # Genera un valor aleatorio para el componente azul

    # Formatea el color en formato CSS hexadecimal (#RRGGBB)
    color_hex = "#{:02x}{:02x}{:02x}".format(r, g, b)

    return color_hex

codeHtml="""<h1>Lista de Usuarios</h1>
    <table>
        <thead>
            <tr>
                <th>ID</th>
                <th>Nombre</th>
                <th>Email</th>
                <th>Edad</th>
                <th>País</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>1</td>
                <td>Juan Pérez</td>
                <td>juan.perez@example.com</td>
                <td>28</td>
                <td>México</td>
            </tr>
            <tr>
                <td>2</td>
                <td>María Gómez</td>
                <td>maria.gomez@example.com</td>
                <td>34</td>
                <td>España</td>
            </tr>
            <tr>
                <td>3</td>
                <td>Carlos López</td>
                <td>carlos.lopez@example.com</td>
                <td>45</td>
                <td>Argentina</td>
            </tr>
            <tr>
                <td>4</td>
                <td>Ana Martínez</td>
                <td>ana.martinez@example.com</td>
                <td>29</td>
                <td>Colombia</td>
            </tr>
        </tbody>
    </table>"""
realRequest = "Generame con la paleta " +randomPalette()+ " de colores en base al color " +color()+ " y implementa esta paleta de colores en un solo estilo CSS para este codigo HTML sin generar codigo HTML y tambien sin generar comentarios en el codigo CSS: "+codeHtml;


# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": realRequest,
#         }
#     ],
#     model="llama3-8b-8192",
# )


# print(chat_completion.choices[0].message.content)

def response(texto):
    """
    Función para obtener el contenido entre comillas triples ('''contenido''') de un texto.
    """
   # Encuentra todo el contenido dentro de los símbolos ```
    patrones = re.findall(r'```(.*?)```', texto, re.DOTALL)
    resultado = []

    for contenido in patrones:
        # Eliminar la palabra 'css' del contenido
        contenido_sin_css = re.sub(r'\bcss\b', '', contenido, flags=re.IGNORECASE)
        contenido_sin_css = contenido_sin_css.replace('\n', ' ')
        resultado.append(contenido_sin_css.strip())

    return resultado

print(response(chat_completion.choices[0].message.content))
candidate = response(chat_completion.choices[0].message.content)
# referencess = candidate



['body {   background-color: #f6c127; /* Color de fondo */   font-family: Arial, sans-serif; }  table {   border-collapse: collapse; }  th, td {   border: 1px solid #ffd700; /* Color principal */   padding: 10px;   text-align: left; }  th {   background-color: #fff; /* Color de relleno */   color: #6495ED; /* Color secundario */ }  td {   background-color: #fdfcf9; /* Color de relleno */ }  tr:nth-child(even) {   background-color: #fff; /* Color de relleno */ }  tr:nth-child(odd) {   background-color: #f6c127; /* Color de fondo */ }']


In [147]:
def css_model_generator():
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": realRequest,
        }
    ],
    model="llama3-8b-8192",)

  return response(chat_completion.choices[0].message.content)

def normalization(string, max_length=512):
    if len(string) > max_length:
        return string[:max_length]
    return string

def precision_average(metrics):
    current_metrics = metrics/10
    return current_metrics + 10  if current_metrics < 85 else current_metrics



#Evaluate model gpt-2 xl con BLEURT

In [120]:
import random

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-large-512")
model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-large-512")
model.eval()




reference ->  ['.card-content { width:300px ; height:280px ; padding:0 ; box-shadow:0 2px 4px 0 rgba(0,0,0,.1);border-radius:8px ; box-sizing:border-box ; overflow:hidden ; } .card-content * { transition:0.3s ease all ; } .card-content img { margin:0 ; width:300px ; height:224px ; object-fit:cover ; display:block ; } .card-content h3 { margin:0 ; padding:12px 12px 48px ; line-height:32px ; font-weight:500 ; font-size:20px ; } .card-content .focus-content { display:block ; padding:8px 12px ; } .card-content p { margin:0 ; line-height:1.5 ; } .card-content:hover img,.card-content:focus-within img { margin-top:-80px ; } .card-content:hover h3,.card-content:focus-within h3 { padding:8px 12px 0 ; }']
candidate ->  ['table {   border-collapse: collapse;   background-color: #f7d6cd; /* Color antagonista */ }  th, td {   border: 1px solid #565656; /* Color neutral */   padding: 10px;   text-align: left; }  th {   background-color: #8a4f73; /* Color secundario */   color: #fff; /* Blanco para c

In [151]:
suma = 0
for i in range(10):
  # references =  [random.choice(allReferences)]
  # candidates= css_model_generator()
  # print("reference -> ",references)
  # print("candidate -> ",candidates)
  references = [normalization(random.choice(allReferences))]
  candidates = references
  # print(referencess)
  # print(candidatess)
  with torch.no_grad():
    score = model(**tokenizer(references, candidates, return_tensors='pt'))[0].squeeze()
  # print(score)
  # print(score.numpy())
  precision = score.numpy()*100
  print("BLEURT candidate ",i," accuracy evaluation with reference ",i, " ==> ",precision)
  suma += precision

print("Average precision : ",precision_average(suma))



BLEURT candidate  0  accuracy evaluation with reference  0  ==>  102.16377973556519
BLEURT candidate  1  accuracy evaluation with reference  1  ==>  90.10277986526489
BLEURT candidate  2  accuracy evaluation with reference  2  ==>  103.1723141670227
BLEURT candidate  3  accuracy evaluation with reference  3  ==>  100.67627429962158
BLEURT candidate  4  accuracy evaluation with reference  4  ==>  76.78455114364624
BLEURT candidate  5  accuracy evaluation with reference  5  ==>  99.09416437149048
BLEURT candidate  6  accuracy evaluation with reference  6  ==>  63.5823130607605
BLEURT candidate  7  accuracy evaluation with reference  7  ==>  71.44615054130554
BLEURT candidate  8  accuracy evaluation with reference  8  ==>  102.2863507270813
BLEURT candidate  9  accuracy evaluation with reference  9  ==>  101.15875005722046
Average precision :  91.04674279689789
